In [3]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import bs4
import requests
from pprint import pprint
import Memory_Collaborative_Filtering as mem
import sqlite3 as sql

In [ ]:
def url_builder_1(book_title):
    path = 'https://isbndb.com/search/books/'
    title_list = book_title.split()
    final_path_list=[]
    for x in range(len(title_list)):
        if x==len(title_list)-1:
            final_path_list.append(title_list[x])
        else:
            final_path_list.append(title_list[x]+'%20')
            
    return path+''.join(final_path_list)

In [ ]:
def url_builder_2(isbn13):
    path = "https://isbndb.com/book/"
    return path+isbn13

In [ ]:
def retrieve_page_content(url):
    page = requests.get(url)
    return page.content

In [ ]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, bs4.element.Comment):
        return False
    return True


def text_from_html(body):
    soup = bs(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

In [ ]:
def retrieve_author(book_title, product_id, html_text, index):
    author='Cannot Locate'
    html_text_list = html_text.split()
    for x in range(len(html_text_list)):
        if html_text_list[x]=='Authors:':
            author=html_text_list[x+1]+' '+html_text_list[x+2]
            break
    author_dict = {'book_title': book_title, 'product_id': product_id, 'author': author}
    author_table = pd.DataFrame(author_dict, index=[index])
    return author_table

In [ ]:
def retrieve_isbn13(book_title, product_id, html_text, index):
    isbn13 = 'Cannot Locate'
    html_text_list = html_text.split()
    for x in range(len(html_text_list)):
        if html_text_list[x]=='ISBN13:':
            isbn13= html_text_list[x+1]
            break
    isbn13_dict = {'book_title': book_title, 'product_id':product_id, 'ISBN13':isbn13}
    isbn13_table = pd.DataFrame(isbn13_dict, index=[index])
    return isbn13_table

In [ ]:
def main_author(data):
    final_authors=pd.DataFrame()
    counter=0
    for x in range(len(data)):
        try:
            counter+=1
            url=url_builder_1(data['product_title'][x])
            html=retrieve_page_content(url)
            text_html=text_from_html(html)
            author = retrieve_author(data['product_title'][x], data['product_id'][x], text_html, counter)
            final_authors = final_authors.append(author)
            print(counter)
        except:
            pass
    return final_authors

In [ ]:
def retrieve_book_category(book_title, product_id, html_text, index):
    book_category = 'Cannot Locate'
    html_text_list = html_text.split()
    for x in range(len(html_text_list)):
        if html_text_list[x]=='Subjects':
            book_categories = html_text_list[x+1]+', '+html_text_list[x+2]+', '+html_text_list[x+3]
            break
    book_categories_dict = {'book_title': book_title, 'product_id':product_id, 'book_category':book_categories}
    book_categories_table = pd.DataFrame(book_categories_dict, index=[index])
    return book_categories_table

In [ ]:
def main_categories(data):
    counter=0
    book_categories=pd.DataFrame()
    for x in range(len(data)):
        try:
            counter+=1
            url=url_builder_1(data['product_title'][x])
            html=retrieve_page_content(url)
            text_html=text_from_html(html)
            isbn13 = retrieve_isbn13(data['product_title'][x], data['product_id'][x], text_html, counter)
            url2=url_builder_2(isbn13.iloc[0]['ISBN13'])
            html2=retrieve_page_content(url2)
            text_html2=text_from_html(html2)
            
            category=retrieve_book_category(data['product_title'][x], data['product_id'][x], text_html2, counter)
            book_categories = book_categories.append(category)
            print(counter)
        except:
            pass
    return book_categories